In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random
import os
import pickle

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

In [3]:
#import eif

In [4]:
random.seed(42)
np.random.seed(42)

## Data Preparation

- Load the data

In [5]:
df = pd.read_parquet('OLDTv3 Datasets')
df.set_index('hashed_tln', inplace=True)

df.head()

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

- Remove CO, META_for_month, Weather
- FillNA with 0

In [ ]:
data = df.drop([c for c in df.columns if 'weather' in c], axis=1)


In [ ]:
data.fillna(data.mean(),inplace=True)

In [ ]:
data.count().isnull()

## Split dataframes by year 2020 and 2021

In [ ]:
df1 = data[data['META_for_month'] <= '2020-12']
print(df1)

In [ ]:
df2 = data[data['META_for_month'] >= '2021-01']
print(df2)

In [ ]:
df1.drop('META_for_month', axis=1, inplace=True)
df2.drop('META_for_month', axis=1, inplace=True)

## Train-Val-Test Split

In [ ]:
RANDOM_SEED = 42

Original Split

In [ ]:
def getCounts(df, col='is_impacted'):
    df_counts = df[col].value_counts().to_frame()
    df_counts['pct'] = df[col].value_counts(normalize=True)
    return df_counts

In [ ]:
getCounts(data)

In [ ]:
impacted = df1[df1.is_impacted==1]
not_impacted  = df1[df1.is_impacted==0]

In [ ]:
# split the impacted data, 50-50 
#impacted_val, impacted_test = train_test_split(impacted,
                                               #test_size=0.5,
                                               #shuffle=True,
                                               #random_state=RANDOM_SEED)

In [ ]:
# split the not_impacted data, the test+val size is equal to the count of impacted
train, val = train_test_split(df1,
                                                        test_size=0.4,
                                                        shuffle =True,
                                                        random_state=RANDOM_SEED, stratify=impacted)
# split not_impacted_val to get a test set
# val, test = train_test_split(val,
                                                     #  test_size=0.5,
                                                     #  shuffle=True,
                                                     # random_state=RANDOM_SEED)

In [ ]:
# combine the impacted and not_impacted data
#train_data = not_impacted_train
#val_data = impacted_val.append(not_impacted_val)
#test_data = impacted_test.append(not_impacted_test)

New Splits

In [ ]:
getCounts(train)

In [ ]:
getCounts(val)

In [ ]:
#getCounts(test)

Separate the Features and Target variables

In [ ]:
X_train = train.drop('is_impacted', axis=1)
y_train = train['is_impacted']

X_val = val.drop('is_impacted', axis=1)
y_val = val['is_impacted']

X_test = df2.drop('is_impacted', axis=1)
y_test = df2['is_impacted']

## Modelling

Scale and Apply PCA

In [ ]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
def get_min_pcs(X, n=0.99):
    
    pca = PCA(svd_solver='full')
    new_X2 = pca.fit_transform(X)
    
    var_explained = pca.explained_variance_ratio_
    
    fig, ax = plt.subplots(1, 2, figsize=(16,6))
    ax[0].plot(np.arange(1, len(var_explained)+1), var_explained)
    ax[0].set_xlabel('PC')
    ax[0].set_ylabel('variance explained')
    
    cum_var_explained = var_explained.cumsum()
    ax[1].plot(np.arange(1, len(cum_var_explained)+1),
                  cum_var_explained, '-o')
    ax[1].set_ylim(bottom=0)
    ax[1].set_xlabel('PC')
    ax[1].set_ylabel('cumulative variance explained');
    
    return np.searchsorted(cum_var_explained, n) + 1

In [ ]:
min_pcs = get_min_pcs(X_train_scaled)
print(min_pcs)

In [ ]:
pca = PCA(n_components=min_pcs, svd_solver='full')
X_train_pca = pca.fit_transform(X_train_scaled)

### One Class SVM

Helper Functions

In [ ]:
def plotConfMat(clf, X, y, **kwargs):
    y_pred = clf.predict(X)
    
    # map predictions to 0, 1
    if 'mapper' in kwargs:
        y_pred = kwargs['mapper'](y_pred)
    
    # plot the confusion matrix
    confmat = confusion_matrix(y_true=y, y_pred=y_pred)
    
    fig, ax = plt.subplots(figsize=(4,4))
    ax.matshow(confmat, cmap='Blues', alpha=0.3)
    
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('Actual Label')
    ax.grid(False)
    ax.vlines(x=0.5, ymin=-0.5, ymax=1.5, color=(0.8, 0.8, 0.8))
    ax.hlines(y=0.5, xmin=-0.5, xmax=1.5, color=(0.8, 0.8, 0.8))
    
    # design
    if 'title' in kwargs:
        fig.suptitle(kwargs['title'], )
        print(kwargs['title'])
        
    if 'ticklabels' in kwargs:
        ticklabels = kwargs['ticklabels']
        ax.set_xticklabels(['']+ticklabels)
        ax.set_yticklabels(['']+ticklabels)
        print(classification_report(y, y_pred, target_names=kwargs['ticklabels']))
    else:
        print(classification_report(y, y_pred))

    plt.tight_layout()
    
    return fig

In [ ]:
def mapPreds(x):
    if x == -1:
        return 1
    elif x == 1:
        return 0
    else:
        return x
    
mapPreds = np.vectorize(mapPreds)

In [ ]:
clf_svm = OneClassSVM(gamma='scale', nu=0.05, verbose=True)
svm_model = Pipeline([('scaler', scaler), ('pca', pca), ('clf', clf_svm)], verbose=True)
svm_model.fit(X_train)

In [ ]:
# plotConfMat(svm_model, X_train, y_train, title='OneClassSVM Train Set')
plotConfMat(svm_model, X_val, y_val,
            title='OneClassSVM Validation Set', mapper=mapPreds,
            ticklabels=['Not Impacted', 'Impacted']);

In [ ]:
os.makedirs('models', exist_ok=True)

with open('models/oneClassSVM.pkl', 'wb') as fp:
    pickle.dump(svm_model, fp)

### Isolation Forest

In [ ]:
iso_forest = IsolationForest(random_state=RANDOM_SEED, contamination=0.03, n_jobs=-1, verbose=True)
if_model = Pipeline([('scaler', scaler), ('pca', pca), ('clf', iso_forest)], verbose=True)
if_model.fit(X_train)

In [ ]:
with open('models/iso_forest.pkl', 'wb') as fp:
    pickle.dump(if_model, fp)

In [ ]:
plotConfMat(if_model, X_val, y_val,
            title='Isolation Forest Validation Set', mapper=mapPreds,
            ticklabels=['Not Impacted', 'Impacted']);
plt.tight_layout()

### Extended Isolation Forest Level 1

In [ ]:
ext_iso_forest_lvl1 = eif.iForest(X_train.values, ntrees=100, sample_size=256, ExtensionLevel=1)

We define the threshold using the anomaly scores

In [ ]:
anomaly_scores = ext_iso_forest_lvl1.compute_paths(X_in=X_val.values)

In [ ]:
df_yval = y_val.to_frame()
df_yval['anomaly_score_pred'] = anomaly_scores
df_yval.head()

Since EIF does not return a prediction, we create our own threshold using the validation set

We scale the anomaly scores using y_val (sklearn precision_recall_curve accepts 0-1 range) then plot the precision-recall curve to get the threshold

In [ ]:
yval_scaler = MinMaxScaler()

yval_anomaly_scaled = yval_scaler.fit_transform(df_yval.anomaly_score_pred.values.reshape(-1, 1))

In [ ]:
df_yval['impacted_proba'] = yval_anomaly_scaled
df_yval['not_impacted_proba'] = 1-yval_anomaly_scaled

In [ ]:
p, r, t = precision_recall_curve(y_true=df_yval.is_impacted, probas_pred=df_yval['impacted_proba'])

In [ ]:
fig, ax = plt.subplots()

ax.step(x=r, y=p)
ax.set_facecolor('white')
ax.set(xlabel='Recall',
       ylabel='Precision',
       title='Precision-Recall Curve for Extended Isolation Forest Level 1 - Val Set Predictions')

ax.grid(True, linewidth=0.2)

inverse transform the threshold

In [ ]:
thresh_idx = np.argmax(p[:-203])
print(f'Optimal Precision {p[thresh_idx]:.02%}')
print(f'Recall @ Optimal Precision {r[thresh_idx]:.02%}')

anomaly_thresh = yval_scaler.inverse_transform(np.array(t[thresh_idx]).reshape(1,-1))
print(f'Anomaly Score Threshold {anomaly_thresh[0,0]:.04f}')

Plot Threshold for Optimal Precision

In [ ]:
fig, ax = plt.subplots()

sns.histplot(data=df_yval.loc[df_yval.is_impacted==0, 'anomaly_score_pred'],
             ax=ax,
             color='blue',
             binwidth=0.01,
             label='Not Impacted')
sns.histplot(data=df_yval.loc[df_yval.is_impacted==1, 'anomaly_score_pred'],
             ax=ax, color='orange',
             binwidth=0.01,
             label='Impacted')
ax.vlines(x=anomaly_thresh[0,0], ymin=0, ymax=600, ls='--', colors='red')
ax.set_ylim(0, 600)
ax.grid(lw=0.2)
ax.legend(facecolor='white')
ax.set_facecolor('white')

For Threshold at Max Precision

In [ ]:
df_yval['y_pred'] = df_yval.anomaly_score_pred.apply(lambda x: 1 if x > anomaly_thresh else 0)

In [ ]:
def plotConfMat_EIF(y_true, y_pred, **kwargs):
    # plot the confusion matrix
    confmat = confusion_matrix(y_true=y_true, y_pred=y_pred)
    
    fig, ax = plt.subplots(figsize=(4,4))
    ax.matshow(confmat, cmap='Blues', alpha=0.3)
    
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('Actual Label')
    ax.grid(False)
    ax.vlines(x=0.5, ymin=-0.5, ymax=1.5, color=(0.8, 0.8, 0.8))
    ax.hlines(y=0.5, xmin=-0.5, xmax=1.5, color=(0.8, 0.8, 0.8))
    
    # design
    if 'title' in kwargs:
        fig.suptitle(kwargs['title'], )
        print(kwargs['title'])
        
    if 'ticklabels' in kwargs:
        ticklabels = kwargs['ticklabels']
        ax.set_xticklabels(['']+ticklabels)
        ax.set_yticklabels(['']+ticklabels)
        print(classification_report(y_true, y_pred, target_names=kwargs['ticklabels']))
    else:
        print(classification_report(y_true, y_pred))

    plt.tight_layout()
    
    return fig

In [ ]:
plotConfMat_EIF(df_yval.is_impacted, df_yval.y_pred, 
                title='Extended Isolation Forest Level 1 Validation Set - Max Precision', 
                ticklabels=['Not Impacted', 'Impacted']);

In [ ]:
with open('models/extended_isoForest_1_p.pkl', 'wb') as fp:
    pickle.dump({'model':ext_iso_forest_lvl1, 'threshold':anomaly_thresh[0,0]}, fp)

In [ ]:
#with open('models/extended_isoForest_1_r.pkl', 'wb') as fp:
   # pickle.dump({'model':ext_iso_forest_lvl1, 'threshold':anomaly_thresh_r[0,0]}, fp)

**Predict on the Test Set**

- We can now use the threshold taken above, no need to scale the anomaly scores

For Threshold at Optimal Precision

In [ ]:
df_ytest = y_test.to_frame()

test_anomaly_scores = ext_iso_forest_lvl1.compute_paths(X_in=X_test.values)

df_ytest['anomaly_score_pred'] = test_anomaly_scores
df_ytest['y_pred'] = df_ytest.anomaly_score_pred.apply(lambda x: 1 if x > anomaly_thresh else 0)

In [ ]:
plotConfMat_EIF(df_ytest.is_impacted, df_ytest.y_pred, 
                title='Extended Isolation Forest Level 1 Test Set - Max Precision', 
                ticklabels=['Not Impacted', 'Impacted']);

**Extended Isolation Forest Max Extended Level**

In [ ]:
ext_iso_forest_maxLvl = eif.iForest(X_train.values, ntrees=100, sample_size=256, ExtensionLevel=min_pcs-1)

In [ ]:
anomaly_scores = ext_iso_forest_maxLvl.compute_paths(X_in=X_val.values)

df_yval = y_val.to_frame()
df_yval['anomaly_score_pred'] = anomaly_scores
df_yval.head()

In [ ]:
yval_scaler = MinMaxScaler()

yval_anomaly_scaled = yval_scaler.fit_transform(df_yval.anomaly_score_pred.values.reshape(-1, 1))

df_yval['impacted_proba'] = yval_anomaly_scaled
df_yval['not_impacted_proba'] = 1-yval_anomaly_scaled

In [ ]:
p, r, t = precision_recall_curve(y_true=df_yval.is_impacted, probas_pred=df_yval['impacted_proba'])

fig, ax = plt.subplots()

ax.step(x=r, y=p)
ax.set_facecolor('white')
ax.set(xlabel='Recall',
       ylabel='Precision',
       title='Precision-Recall Curve for Extended Isolation Forest Max Level - Val Set Predictions')

ax.grid(True, linewidth=0.2)

inverse transform the threshold

In [ ]:
thresh_idx = np.argmax(p[:-228])
print(f'Optimal Precision {p[thresh_idx]:.02%}')
print(f'Recall @ Optimal Precision {r[thresh_idx]:.02%}')

anomaly_thresh = yval_scaler.inverse_transform(np.array(t[thresh_idx]).reshape(1,-1))
print(f'Anomaly Score Threshold {anomaly_thresh[0,0]:.04f}')

Plot Threshold for Optimal Precision

In [ ]:
fig, ax = plt.subplots()

sns.histplot(data=df_yval.loc[df_yval.is_impacted==0, 'anomaly_score_pred'],
             ax=ax,
             color='blue',
             binwidth=0.01,
             label='Not Impacted')
sns.histplot(data=df_yval.loc[df_yval.is_impacted==1, 'anomaly_score_pred'],
             ax=ax, color='orange',
             binwidth=0.01,
             label='Impacted')
ax.vlines(x=anomaly_thresh[0,0], ymin=0, ymax=600, ls='--', colors='red')
ax.set_ylim(0, 600)
ax.grid(lw=0.2)
ax.legend(facecolor='white')
ax.set_facecolor('white')

For Threshold at Optimal Precision

In [ ]:
df_yval['y_pred'] = df_yval.anomaly_score_pred.apply(lambda x: 1 if x > anomaly_thresh else 0)

plotConfMat_EIF(df_yval.is_impacted, df_yval.y_pred, 
                title='Extended Isolation Forest Max Level Validation Set - Max Precision', 
                ticklabels=['Not Impacted', 'Impacted']);

In [ ]:
#with open('models/extended_isoForest_max_p.pkl', 'wb') as fp:
   # pickle.dump({'model':ext_iso_forest_maxLvl, 'threshold':anomaly_thresh[0,0]}, fp)

In [ ]:
#with open('models/extended_isoForest_max_r.pkl', 'wb') as fp:
  #  pickle.dump({'model':ext_iso_forest_maxLvl, 'threshold':anomaly_thresh_r[0,0]}, fp)

**Predict on Test Set**

For Threshold at Optimal Precision

In [ ]:
df_ytest = y_test.to_frame()

test_anomaly_scores = ext_iso_forest_maxLvl.compute_paths(X_in=X_test.values)

df_ytest['anomaly_score_pred'] = test_anomaly_scores
df_ytest['y_pred'] = df_ytest.anomaly_score_pred.apply(lambda x: 1 if x > anomaly_thresh else 0)

In [ ]:
plotConfMat_EIF(df_ytest.is_impacted, df_ytest.y_pred, 
                title='Extended Isolation Forest Max Level Test Set - Max Precision', 
                ticklabels=['Not Impacted', 'Impacted']);